In [6]:
import matplotlib.pyplot as plt
import PIL
import PIL.Image as Image
import PIL.ImageDraw2 as ImageDraw
import io
import tensorflow as tf

In [4]:
file='PathToMyTFrecord'
rawdata=tf.data.TFRecordDataset(file)

In [10]:
for record,rawrecord in enumerate(rawdata.as_numpy_iterator()):
    example=tf.train.Example()
    example.ParseFromString(rawrecord)
    rawdict=dict(example.features.feature)
    
    imgbytes=rawdict['image/encoded'].bytes_list.value[0]
    img=Image.open(io.BytesIO(imgbytes))
    
    width=rawdict['image/width'].int64_list.value[0]
    height=rawdict['image/height'].int64_list.value[0]
    print(f"Overlaying Annotations for Image:{rawdict['image/filename'].bytes_list.value[0].decode()}")
    for idx,tag in enumerate(rawdict['image/object/class/text'].bytes_list.value):
        minxy=(int(rawdict['image/object/bbox/xmin'].float_list.value[idx]*width),
               int(rawdict['image/object/bbox/ymin'].float_list.value[idx]*height))
        maxxy=(int(rawdict['image/object/bbox/xmax'].float_list.value[idx]*width),
               int(rawdict['image/object/bbox/ymax'].float_list.value[idx]*height))
        img_overlay=PIL.ImageDraw.Draw(img)
        img_overlay.rectangle([minxy,maxxy],width=3)
    img.save(rawdict['image/filename'].bytes_list.value[0].decode())